# Fasttext Model Train
- Fasttext 모델을 판례 데이터로 학습

## 1. 패키지 로드

In [1]:
import pandas as pd
import numpy as np
import re
import warnings

from konlpy.tag import Mecab
from gensim.models import FastText
from nltk import sent_tokenize
from sqlalchemy import create_engine
from tqdm import tqdm_notebook

In [2]:
# 경고무시를 위한 코드
warnings.filterwarnings('ignore')

## 2. 판례 데이터 로드
> 최신판례가 DB에 저장되어있기 때문에 DB로부터 데이터를 가져온다.

### 1) DB 정보

In [3]:
host = 'localhost'
user = 'root'
password = '1234'
db = 'cases'
port = 3306

### 2) DB 연동

In [4]:
# 1. DB 연동
db_connection_str = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
db_connection = create_engine(db_connection_str)
connect = db_connection.connect()

### 3) 데이터 읽기

In [5]:
df = pd.read_sql("SELECT * FROM case_info", connect)
df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,64437,사해행위취소등,2003나1746,2003-07-10,선고,대전고법,4002,민사,4001,판결: 상고,파산자의 채권에 기한 사해행위취소의 소에서 사해행위 사실을 알았는지 여부를 파산관재...,파산법이 파산관재인에게 파산재단에 관한 소에 있어 원고 또는 피고가 된다고 한 것은...,"민법 제406조 제2항\n,\n\n 파산법 제7조","\n대법원 1990. 11. 13. 선고 88다카26987 판결(공1991, 57)","【원고,항소인】 파산자 주식회사 충남상호신용금고의 파산관재인 예금보험공사 (소송..."
1,64438,위증,2008도3300,2008-06-26,선고,대법원,4002,형사,4001,판결,[1] 공범인 공동피고인이 다른 공동피고인에 대한 공소사실에 관하여 증인적격이 있는...,[1] 공범인 공동피고인은 당해 소송절차에서는 피고인의 지위에 있으므로 다른 공동피...,"[1] \n \n 형사소송법 제146조, \n ...",[1] \n \n 대법원 1983. 10. 25. 선고 8...,【피 고 인】 【상 고 인】 검사【원심판결】 수원지법 2008. 4. 1. 선고 2...
2,64439,상고장각하명령에대한이의,2007무77,2008-06-02,자,대법원,4002,일반행정,4001,결정,[1] 소송구조신청에 대한 기각결정이 확정되기 전에 소장 등에 인지를 첩부하지 않았...,[1] \n \n 행정소송법 제8조 제2항에 의하여...,"[1] \n \n 민사소송 등 인지법 제1조, \n ...",[1] \n \n 대법원 1993. 1. 25.자 92마1...,【재항고인(선정당사자)】 【상 대 방】 인천광역시장【원심명령】 서울고법 2007. ...
3,64440,손해배상(기),2006다49284,2008-09-25,선고,대법원,4002,민사,4001,판결,광산에서 금광의 탐광 및 채광을 위한 굴진공사를 계속 진행할 경우 인근 주민들에게 ...,미기재,"민법 제214조, \n 제217조, \n 헌법 제35조",없음,"【원고, 피상고인 겸 상고인】 【피고, 상고인 겸 피상고인】 【원심판결】 대전고법 ..."
4,64441,임대차보증금등·지료등,"2006다62492,62508",2008-09-25,선고,대법원,4002,민사,4001,판결,"[1] 계약의 법정 또는 약정 해지사유 발생시, 당사자가 경매신청 등 계약해지를 전...",미기재,"[1] \n 민법 제105조, \n 제543조 / [2] ...",없음,"【원고(반소피고), 상고인 겸 피상고인】 【피고(반소원고), 피상고인 겸 상고인】 ..."


## 3. 전처리

### 1) 문장 토큰화 전 전처리
- 특수문자, 숫자 등은 문장 토큰화시에 방해요인이므로 제거한다.

In [7]:
df['X'] = df['판례내용'].str.replace('[^가-힣. ]', '')
df['X'].head(3)

0    원고항소인   파산자 주식회사 충남상호신용금고의 파산관재인 예금보험공사 소송대리인 ...
1    피 고 인 상 고 인 검사원심판결 수원지법 . . . 선고 노 판결주    문  상...
2    재항고인선정당사자 상 대 방 인천광역시장원심명령 서울고법 . . .자 누 병합 상고...
Name: X, dtype: object

### 2) 문장 토큰화
- 각 판례 본문을 문장으로 토큰화한다.

In [8]:
sentence_tokened = []

for x in tqdm_notebook(df['X']):
    sentence_tokened.extend(sent_tokenize(x))

  0%|          | 0/100 [00:00<?, ?it/s]

### 3) 형태소 토큰화 전 전처리
- 형태소 토큰화 전 한글 외의 문자들은 모두 제거한다.

In [10]:
clean_sentence_tokened = []
for sent in tqdm_notebook(sentence_tokened):
    clean_sentence_tokened.append(re.sub('[^가-힣 ]', '', sent))

  0%|          | 0/2653 [00:00<?, ?it/s]

### 4) 형태소 토큰화

#### (1) Mecab 형태소 분석기

In [11]:
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

#### (2) 형태소 토큰화

In [12]:
tokenized_vocab = []

for sent in tqdm_notebook(clean_sentence_tokened):
    token_word = mecab.morphs(sent)
    tokenized_vocab.append(token_word)

  0%|          | 0/2653 [00:00<?, ?it/s]

## 4. 학습

In [17]:
model = FastText(tokenized_vocab, vector_size=512, window=7, min_count=5, workers=4)

## 5. 모델 저장

In [18]:
model.save('fasttext.model')